## Text generation with LSTM-Recurrent Networks

Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.

In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using TensorFlow backend.


In [2]:
path = 'data/nietzsche/nietzsche.txt'
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 600893


In [11]:
text[:5]

'prefa'

In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


In [4]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 200285


In [5]:
sample_sentences = sentences[:20000]

In [6]:
print('Vectorization...')
x = np.zeros((len(sample_sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sample_sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sample_sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [7]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

Build model...


In [8]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [9]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [10]:
# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1
Epoch 1/1
20000/20000 [==============================] - 59s 3ms/step - loss: 3.2905

----- diversity: 0.2
----- Generating with seed: "ent, but
finally some spectator, inspire"
ent, but
finally some spectator, inspireiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii

----- diversity: 0.5
----- Generating with seed: "ent, but
finally some spectator, inspire"
ent, but
finally some spectator, inspireiiiiiiiiii ieiiiiiiiiietiiiiiieiiiiiiiiiiiii
ieiii
iiiiiiiiioiiiiieiiiiiii iiiiiiiiieiiiiieiiitiie
i
ifiiiiiiiiiiiiiiiiiiiiiiiiiiitiiiiiiiiiiiieiiiiiiiieii
eiiieiiiiiiiiioiiiiiiiiiiiii

reasoning from effect back to cause. asnneeennefneniseanaonsgnneehncne.ernne eenwsdncytnen senncnnnnrngnawnnstnteynnnncnasssnhsee nnnhceeennenc neenrnet nnsnnnngnnw ennhsndedaeneeeaar
egmttaedennwnfnnsns snsennnnsnrer:nnsnnnnsennsaasnntnhnn:qnseqrnn,seutesenrenrnn nseheosnnnnnnenttr,ennennsrennsaiennkstrn nnsenrnesnntnnntnantn'shennenoeonarqnennnnnesen sntnennnisrenaenraeneerr,en,hnennnnewnstaennessenenalaqnrnnnrntnnnnnsenqhnsnsnnen nn

----- diversity: 1.2
----- Generating with seed: "
reasoning from effect back to cause. as"

reasoning from effect back to cause. asnawnnusneenensnaewle mnn  nnneeeneannsntnassebneornnnsrrnnnge
srayerrnebnsn, s:hw,ancnnnnuerdcaeareennnucenneenanennrtgnietssnsa seiennnseana,,nnnr
easeenrn
nnconsnnesnnsccnstnoneeewqnnqewnns ieere-lcngn,eman,,nnnstcetbg,qsesnclti nenepnsetngennrn: qenrwe:nerunnensfornannnhnneerwetneensc,n,enna: nms
asnsnniswnnnwgnrlnennnsn,nnnae n
nnyoosraetesndrnnasnhentmrgcntxsnsonreaaee,n,rwetssninn snsfntenow

-------------------------

their power: even the hshseehhhlshehhheheshehlheeehheehhsssssehssshh hshheelshheeehsehhhheessehhheeeeehhhhhlhheehhhhhehhs shhhehleeeheehshhehhhehhhhhhehhhhhhhehehhheeeehh ehslsehhehhehhhhhehehhheleeehhhlehhehhhhshhhhhhhheseehhe hhehhhhhhlehhehehhheehsehlehlhhhehhhshehshhlhelesheheheelhslehehhhhhhhehhe heheehehhhheeeheeehlhhhhhhehhehehseeeehhhheehheeehh hehhehheeehhshhhehshhhehhlhhhhhhehhseeshehhhhlehhesehheehh hheelehh

----- diversity: 0.5
----- Generating with seed: " anew the prop of
their power: even the "
 anew the prop of
their power: even the sr hheehhr sesheebhs  hehwhels heeehehhehhsl shahhhhshrellseehehshhweieerhneshhhshfslelehsohhhs ellh errl,hph llsheeeesshehssh hshhheelhhhshrhbseh hh hrhelhseelhslrobrshhheahsilhllehw s eshhheh l hhhlhhehhsehssh sehhsess hlhhishhhserplhe slershhh hplsehhhe,s,e,shlhee,hhehlhshhr shlehthhselesh  ehsrrserhb hlse hhbplheh,hnle,hhsreesslhllhlsehleehhrhhhlbrheslehseeeoe lhrhaehh elehh
hh h lshlserheu ue

----- diversity: 1.0
----- Generating with

tnm t hmmsch"-sl-- " om mp uomu -lds-ssu sn-u-mm-s-y mms elrasadam----i- smymaeshaam   d-n--m- -p slo--m-p--msmro-s-sl"-sdncdoc cosno --oa sedslr -a- msm-m-u-c    ma mssu ass mum-thmocc spaysama

----- diversity: 1.2
----- Generating with seed: "fluence
of such a state of mind, form of"
fluence
of such a state of mind, form of -ue-c-d
-c--yc emrhsns
scn lsp-mdryms--oesdsoomlouom -esm lu ss-wp---w"i
-d s-ssmmf -ems--ss-udmfladouvump-orme
r-b--gmai e-a-s--ms
na-salshf
xe-s-y-e- l s-d acd-npmeobu-m -,ap
cs-lac
m-udl clan
cad -s-;sa-aoanv s-atas-sau-dpash
t-amlhynercunhat-mo-e-s-csse-e-ns  sydmvm  acl  pst-

-ca 
h- sa-u- sflammhcoe- h- la--a tn -mrvescysnt-ayoeoauspmi-osbwm-sas--s-yugeawo-loa-s--p-spmmuwa s
 agmo-m a au- 

--------------------------------------------------
Iteration 12
Epoch 1/1
20000/20000 [==============================] - 50s 3ms/step - loss: 3.2461

----- diversity: 0.2
----- Generating with seed: "f go, and thereby gazes with
faithful, b"
f go, and thereby gazes with

le and pardonable, but they have no righ,es,ssees,hshes,e,se,,s,ss,seeee,,s,ss,,s,,e,se,seesae,,,,ss,ses,esseee,ssss,ss,ssssesss,,ess,ee,sssee,,s,ess,,ss,,,,s,,s,,ssss,se,ss,,s,,es,bssee,e,ess,essee,,s,,,,sss,s,,,sese,ssss,sseseehss,,es,essess,,,sssesssseess,esss,,seeaeseess,ees,,,,eesssesss,s,sse,ss,s,e,seesse,,eeese,s,ese,eee,es,ssssss,,sss,ssese,esesseses,s,,eseseaess,s,ee,ssee,e,,sess,,eh,sssssssssses,eessssss,s,ss,essse,se,,essess,

----- diversity: 0.5
----- Generating with seed: "le and pardonable, but they have no righ"
le and pardonable, but they have no righsshs,eve,a,es,weesses,,,seiseoaaf,she,sevsbetveebvs,e,e,ss,,heoa,v,vsv,eve,eae, sese,s,rehe,eesssee,,es,sees,essesoha,a,,,ee,hsassv,,aavfhyshshehesee,ie,swaseeshes,sevs,ves,seeese,,osa,,,s,eho,se,,whc,,eeohhesa,,,,s,sleesese,,,s,ssssesbs,,thavvheseesesvhea,efi,ssses,ssie,,fssssae,wss,s,sh,w,s,ssosssse,svs,swes,sssnssess,sss,,s,ie,ashssvseehiresvewsfsssevvsse,eeiehfssa,s,s,s,,h,s,s,se,vhsd,s,sresss

----- diversity: 1.0
---

...

e; a man who has his indignation and hispeeiepepeeppeeepeeeeyepepeeiippepee"eeeeeyeeeeeepeeeeeeyieeeeepieeeepeeeeeipeeeepe"eeppeeppeieeppepppeeeieppepeyeeeeeeepppe"eeepeeepepepyeepeqeeeeeeeeeeieeeeipeeeeeeepeeeeeepipepiypepeeyiypepeeeeeiepeeeeeieeepeeeeepppyeepepeeeeyepieyeeiieepepe"ieepeppeyeeippeeeeyeepeipeppiiepieeeppeeepppeeepepeyeipeeepeeeeeepepeeeppeepeepeieeeeeeeeieepeppyeepeeeeeiepeeeee"epepepppnepeiebeieeeypeyeppeeeeeppeyeeeyip

----- diversity: 0.5
----- Generating with seed: "e; a man who has his indignation and his"
e; a man who has his indignation and hisiyepibppepie-iep-neg"eepqppeieey"pyyyeeieeqeqi"eepypspgee"iqippe"eepeebppipiee"egrpeisyeeepyypeepepe"eieeeeiy pynpeqeegippeeiyiqypyei yeye"yepbeiqeeiyeieeese eeiepiiypnieeeqpiypepeqipyyppeieipi p eaipeiepiiiphyyieiyiypepepq ge"yqnie"yiypie"gpeeiieieiepiee"e bgepgppyy-peeeeeeeippeppepebgeqpiyqpeaepyppp"eeeeieybeeeppeiepeegppeeg"yqeepygeeepyyy ygbyieiiieeypeypqeeqyeq"qpee""epypyynyney e"yeqp yeey"ye

----- diversity: 1.0
---

socelnmkeoausatte   s  lcb amappel-anac x iaaaap t  -w  oss ka  c aoselelo   asau- oiausavpels  als s l mtf e k hpsk-uao  popu   a aag b las t   aktekaivkatse s g  u a akksisg pap t us l-x gs?v g aa,a  d e sklkm  eaak ita als s  nh  a bsla ahol ieits ss gat ltttat  la teakskssc  saipuclkbsm      pr ss a eaa tlp  aasuboea  isafaa tt  -       

----- diversity: 1.2
----- Generating with seed: "rm unfavourable
conditions is, as alread"
rm unfavourable
conditions is, as alread!ghu laco oifaluee  tadarapu mtstlikt amlilbufgpa-tm -l- r aa sgims   x  dpei eokcenaset   tc-c nyh  aaess
dlevntciukapcea-   ;  mahee  x imakgassecsse  e savhak  dt  upaaeidalt .ct elld  cgyta matnktlellscm ae-vcs  x xui lanlag-ylaatan   n  lsa k lrtug  dplom elahymeakmlcsllksnvk  klmaeudkg" -p"vrttpcttgu siane"ssvfp sseea  ld-utbg ole aoglawsv ah mmreotasto t goeooelgvk ae al tc!  aat slcei stke

--------------------------------------------------
Iteration 49
Epoch 1/1
20000/20000 [==============================] - 

rious labour of their souls was to prepa               i                                 n         i                        i                                                                                             n                                                                                                                     i                                    i                                                                  

----- diversity: 0.5
----- Generating with seed: "rious labour of their souls was to prepa"
rious labour of their souls was to prepa    pi    i      o i           i i  
    in     i i  i          n   n  ni  i      i  i        i ui  i        i n                        in    t     i   i  n              i   n  i   oi        n i       n  i    iii  i  i i   i      p   i ii  u   i n    p   i   fii                i  ei             n       i    i e i i    i i      i    i          ii  n    ii       i     n i     p     i       on      i

----- diversity: 1.0
---

,l se fdaetd sslne sldtred di cllddsdisddtsetaiesrdrde;sleinsiotcdeds osndsshtd setsddi dsthiosemitdsetddcsdsms.ladinssynddw,dvdasssdohw;nsohfstiedsysdtetdlldceyeatellsmmi udssotseds oiis;cc n.lsns.plln lssee siiostsdt.d idsm;dsisesdteednftclod
sdtidsadsalsdssttlsssrdltdonteos cstdmdesm eises.dlssmsd.litmssitd.cdenddnsidddsfisssds tsndssdss isettntetee clsmecdsssddddeslih itphs
tsleddllledlic.site

----- diversity: 1.2
----- Generating with seed: "tity, like dust. good and bad is, for a
"
tity, like dust. good and bad is, for a
dtpsinndoultdt sditoeiicn ttniidlarthetlddsts sscte"osms ldslnb.ledadndsl;lidd
tsa,dsmdtldssrnadisiselseesd.ogcslsiedlkddodledenstpdo(rmdnibdsst dsossisistltde.scsrdoatddtdddssel;ohsswtn a hasnwdllerdetiudneede nmmeslclders."lsbddydclsoefadc,miitttaai scdeto nmsddidsdettrdsudceot.dailddmdtadesmeashemll!pnsn sllpi"lsdoodm.ssdsliiesasdeutdrgestestds"lhit,rdididaono.t, ncotseenhdddfatdintrseasdi
snsd

--------------------------------------------------
Iteration 56


end, as every means does--spoil all intyy iyyyyyyiyy yyyyyyyiysyyyyyyyyyyysyyyyyyyyyyyyyyyy yyyyyyyyyiyyyyyyyyyyyyyyyyyyyyyyyyyyyyyysyyyyyyyyiiyysyyyyyyyyyyyyyyyyyyyyyyyyyy yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyiyyyyyyyyyyyyyyysyyyyyyiyyy yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy yyyyyyyyyyyyyyyiyyyyyyyyyyyyyyyyssyyy yyyyyiiisysyyyyyyyyyyyyyyyyyyyysyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy

----- diversity: 0.5
----- Generating with seed: "
end, as every means does--spoil all int"

end, as every means does--spoil all intiyioyissyiyhiyiaiysye  yo yyyiyysyssyystyayyshyyihyyyayyiy hyhiayyyyisiiyayyyydyiyy yyhdysys yiyyiiiyihyyyyyye syhis yyasiyyssyyyyshaiyyiyyyyysyhsyhysy hoyyyyyyyysaystiid yyiyi  syiyyyyiayyyaystyeyyy yiyya saiihsiihsyyiyyy yosysy ssyi yyhyosayytayayt yyotyyyoioiiysyohy s iyai yyyahsysyyy yyyasiyiyihiyiyyysyiiyayyyyyyyynh sysyayy ssyyyy ysayyiyiiisyyyyhs ytssa saiiyydyypiyi iyyesyy    ysoyyssayoysi

----- diversity: 1.0
----